<a href="https://colab.research.google.com/github/JuancVela/CUDhack/blob/main/exploration_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Union y exploración de los datasets

In [1]:
import os
import openpyxl
import pandas as pd
import re

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
folder='/content/drive/MyDrive/CUD_hackaton/Data Set hackaton/Data Base'

In [14]:
# Función para leer archivos Excel desde un directorio
def read_excel_files(folder_path, pattern=r'\d+'):
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"The directory {folder_path} does not exist.")

    excel_files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx')]
    dfs = []  # Lista para almacenar DataFrames

    for file_name in excel_files:
        file_path = os.path.join(folder_path, file_name)
        wb = openpyxl.load_workbook(file_path)

        for sheet_name in wb.sheetnames:
            sheet = wb[sheet_name]
            # Solo procesar hojas visibles
            if sheet.sheet_state == 'visible':
                try:
                    df = pd.read_excel(file_path, sheet_name=sheet_name)
                except Exception as e:
                    print(f"Error reading {file_name} - {sheet_name}: {e}")
                    continue
                # Extraer ID del nombre del archivo y limpiar las columnas
                match = re.search(pattern, file_name)
                if match:
                    id_file = str(match.group(0))
                    df['ID'] = id_file
                    df.columns = [col.replace(id_file + "_", "") for col in df.columns]
                dfs.append(df)

    if dfs:
        return pd.concat(dfs, axis=0, ignore_index=True)
    else:
        return pd.DataFrame()  # Retorna un DataFrame vacío si no hay archivos

# Directorio base
#folder = "/Users/juancamilovela/Documents/CUD_hackaton/Data Set hackaton/Data Base"

# Lectura de archivos de Bioreactor
folder_Bioreactor = os.path.join(folder, "Biorreactor")
Bioreactor = read_excel_files(folder_Bioreactor)

# Lectura de archivos de Centrifuga
folder_Centrifuga = os.path.join(folder, "Centrifuga")
Centrifuga = read_excel_files(folder_Centrifuga)

# Lectura de archivos restantes en la carpeta 'Data Base'
excel_files = [file for file in os.listdir(folder) if file.endswith('.xlsx')]

DB = {}  # Diccionario para almacenar DataFrames de cada archivo

for file_name in excel_files:
    file_path = os.path.join(folder, file_name)
    try:
        sheets = pd.read_excel(file_path, sheet_name=None)  # Lee todas las hojas
        for sheet_name, df in sheets.items():
            df_name = f"{file_name}_{sheet_name}"  # Combina el nombre del archivo y de la hoja
            DB[df_name] = df  # Almacena el DataFrame en el diccionario
    except Exception as e:
        print(f"Error reading {file_name}: {e}")

dict_keys(['Fases producción_test.xlsx_Cultivo final', 'Cinéticos IPC.xlsx_Inóculos', 'Cinéticos IPC.xlsx_Cultivos finales', 'Cinéticos IPC.xlsx_Centrifugación', 'Fases producción.xlsx_Preinóculo', 'Fases producción.xlsx_Cultivo final', 'Horas inicio fin centrífugas.xlsx_Hoja1', 'Mov_comp', 'OF', 'TyH_WDato', 'TyH_Datos', 'FP_Inoculo'])


In [21]:
# Mapeo de nombres de claves
key_mappings = {
    'Cinéticos IPC.xlsx_Inóculos': 'C_Inoculo',
    'Cinéticos IPC.xlsx_Cultivos finales': 'C_finales',
    'Cinéticos IPC.xlsx_Centrifugación': 'C_Centrifuga',
    'Fases producción.xlsx_Preinóculo': 'FP_Preinoculo',
    'Fases producción.xlsx_Inóculo': 'FP_Inoculo',
    'Fases producción.xlsx_Cultivo final': 'FP_final',
    'Fases producción_test.xlsx_Cultivo final': 'FP_test_Cf',
    'Horas inicio fin centrífugas.xlsx_Hoja1': 'Hrs_Centrifuga',
    'Movimientos componentes.xlsx_Full1': 'Mov_comp',
    'OF 123456.xlsx_Sheet1': 'OF',
    'Temperaturas y humedades.xlsx_WData': 'TyH_WDato',
    'Temperaturas y humedades.xlsx_Datos': 'TyH_Datos'
    # Añadir más mappings si es necesario
}

for old_key, new_key in key_mappings.items():

    if old_key in DB:

        DB[new_key] = DB.pop(old_key)

# Actualizar el diccionario 'DB' usando los mapeos definidos
#DB = {key_mappings.get(k, k): v for k, v in DB.items()}

# Ver las claves del diccionario actualizado
print(DB.keys())


dict_keys(['Mov_comp', 'OF', 'TyH_WDato', 'TyH_Datos', 'FP_Inoculo', 'FP_test_Cf', 'C_Inoculo', 'C_finales', 'C_Centrifuga', 'FP_Preinoculo', 'FP_final', 'Hrs_Centrifuga'])
